In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import _pickle as cPickle # python3 shit
import re, sys, os, csv
from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import chain
from nltk.classify import NaiveBayesClassifier, accuracy

In [ ]:
stop_words = list(get_stop_words('en'))         #About 900 stop words
nltk_words = list(stopwords.words('english'))   #About 150 stop words
stop_words.extend(nltk_words)

def clean_tweet( tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        tweet = tweet.lower() 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def remove_stopwords(word_list):
        filtered_tweet=""
        for word in word_list:
            word = word.lower() 
            if word not in stopwords.words("english"):
                filtered_tweet=filtered_tweet + " " + word
        
        return filtered_tweet.lstrip()
    
def vectorise_label(label):
    if label == "empty":return 0
    elif label == "sadness":return 1
    elif label == "enthusiasm":return 2
    elif label == "neutral":return 0
    elif label == "worry":return 1
    elif label == "surprise":return 2
    elif label == "love":return 2
    elif label == "fun":return 2
    elif label == "hate":return 3
    elif label == "happiness":return 2
    elif label == "boredom":return 0
    elif label == "relief":return 2
    elif label == "anger":return 3

In [ ]:
data_train = pd.read_csv('dataset/data/text_emotion.csv', sep=',')
print("Dataset shape:",data_train.shape)
print(data_train.sentiment[0],":",data_train.content[0])

In [ ]:
dataWriter = csv.writer(open('data.csv', 'w'), delimiter=',',lineterminator="\n")

total = 40000
for i in range(40000):
    print("Progress: ",round(i/total*100,2),"   ",end="\r")
    tweet= clean_tweet(data_train.content[i])
    tweet = remove_stopwords(tweet.split())
    dataWriter.writerow([tweet, str(vectorise_label(data_train.sentiment[i]))])
    sys.stdout.write("\033[F")
    
print("Progress: ",100,"\nComplete!")

In [3]:
count = 0
with open('data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        count+=1

In [4]:
count

40000